In [1]:
import pandas as pd

In [2]:
import openai
deployment_name = "gpt-4"
openai.api_base = "https://codesidebias.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-08-01-preview"
openai.api_key = "766aeef6717d4c09b997c943f75ce7d5"

In [3]:
from src.utils import get_choices

/data/healthy-ml/scratch/abinitha/miniconda3/envs/detect/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


## Generate clinical LLM agent responses for vignette and conversational formats (multi-turn, single-turn and summarized)

### OpenAI Models (GPT-3.5, GPT-4)

In [4]:
from src.craftmd import craftmd_gpt

In [ ]:
from tqdm import tqdm 

model_name = "gpt4_1106"
dataset = pd.read_csv("/data/healthy-ml/scratch/abinitha/craft-md/craft-md/data_augmented/baseline.csv", index_col=0)

cases = [(dataset.loc[idx,"case_id"], 
          dataset.loc[idx,"case_vignette"], 
          dataset.loc[idx,"category"],
          get_choices(dataset,idx)) for idx in dataset.index]

path_dir = f"results/{model_name}"

for i in tqdm(range(len(dataset))):
    case = cases[i]
    craftmd_gpt(case, path_dir, model_name)

### Open-source Models (Mistral-v1, Mistral-v2, LLaMA2-7b)

These models require GPU resources. We performed all evaluations on Quadro RTX 8000 48gb

In [29]:
from src.craftmd import craftmd_opensource
from src.models import get_model_and_tokenizer

In [15]:
# # To download open-source models, if not already installed in your conda environment
# from huggingface_hub import login
# login(token = "insert_huggingface_token")

In [ ]:
model_name = "mistral-v2"
dataset = pd.read_csv("data/usmle_and_derm_dataset.csv", index_col=0)

cases = [(dataset.loc[idx,"case_id"], 
          dataset.loc[idx,"case_vignette"], 
          dataset.loc[idx,"category"],
          get_choices(dataset,idx)) for idx in dataset.index]

path_dir = f"results/{model_name}"

case = cases[0]

model, tokenizer = get_model_and_tokenizer(model_name)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id 

craftmd_opensource(case, path_dir, model, tokenizer)

### Multimodal LLM (GPT-4V)

In [6]:
from src.craftmd import craftmd_multimodal

In [36]:
model_name = "gpt4v"
dataset = pd.read_csv("data/nejmai_dataset.csv", index_col=0)

cases = [(dataset.loc[idx,"case_id"],
          dataset.loc[idx,"case_vignette"],
          get_choices(dataset,idx)) for idx in dataset.index]

path_dir = f"./results/{model_name}"
img_dir = f"../../craft-md-v2/data/nejm/imgs/"

craftmd_multimodal(cases[0], img_dir, path_dir, deployment_name)

NameError: name 'craftmd_multimodal' is not defined

# Evaluate using Grader-AI agent

Below code works for both open-source and GPT models

In [5]:
from src.graderai_eval import graderai_evaluation

/data/healthy-ml/scratch/abinitha/miniconda3/envs/detect/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

model_name = "gpt4_1106"
dataset = pd.read_csv("data_augmented/baseline.csv", index_col=0)

experiment_names = ["vignette_frq", "multiturn_frq", "singleturn_frq", "summarized_frq"]
path_dir = f"/data/healthy-ml/scratch/abinitha/craft-md/craft-md/results/gpt4_1106/lowercase"

graderai_evaluation("case_10", dataset, path_dir, experiment_names)

Thread for case_10 is dispatched.


In [1]:
import json

file_path = "results/gpt4_1106/baseline/case_2.json"  # Replace with your actual file path

# Read the JSON file
with open(file_path, "r") as f:
    data = json.load(f)  # Load JSON into a dictionary

# Get the list of keys
keys_list = list(data.keys())

# Print the keys
print(keys_list)


['case_vignette', 'question', 'trial_0', 'trial_1', 'trial_2', 'trial_3', 'trial_4', 'correct_ans']


# Evals Code

In [3]:
import json
from pathlib import Path
from collections import defaultdict

def sum_nested_dict(d, sum_dict):
    """Recursively sums numeric values from nested dictionaries."""
    for key, value in d.items():
        if isinstance(value, (int, float)):  # If value is numeric, sum it
            sum_dict[key] += value
        elif isinstance(value, dict):  # If value is a nested dictionary, recurse
            sum_nested_dict(value, sum_dict)

folder_path = Path("results/gpt4_1106/lowercase")  # Replace with actual folder path
count = 0
sum_dict = defaultdict(float)  # Stores the sum of each key

# Iterate over JSON files in the folder
for file_path in sorted(folder_path.glob("*.json")):
    with open(file_path, "r") as f:
        count = count + 1
        data = json.load(f)  # Load JSON into a dictionary

    # Recursively sum values
    sum_nested_dict(data, sum_dict)

# Print the total sum for each key
print(dict(sum_dict))
print(count * 3)

{'evaluation_vignette_frq': 711.0, 'evaluation_multiturn_frq': 433.0, 'evaluation_singleturn_frq': 295.0, 'evaluation_summarized_frq': 400.0}
792


In [ ]:
import json
from pathlib import Path

folder_path = Path("your_folder_path")  # Replace with the actual folder path
key_to_extract = "your_key"  # Replace with the key you want to extract

# Iterate over JSON files in the folder
for file_path in sorted(folder_path.glob("*.json")):  # Sort ensures consistent order
    with open(file_path, "r") as f:
        data = json.load(f)  # Load JSON file into dictionary
    
    # Extract and print the value for the specific key
    if key_to_extract in data:
        print(f"{file_path.name}: {data[key_to_extract]}")
    else:
        print(f"{file_path.name}: Key '{key_to_extract}' not found")


In [10]:
import json
from pathlib import Path

folder_path = Path("/data/healthy-ml/scratch/abinitha/craft-md/craft-md/results/gpt4_1106/whitespace")  # Replace with the actual folder path
keys_to_remove = {"evaluation_vignette_frq", "evaluation_multiturn_frq", 'evaluation_singleturn_frq', 'evaluation_summarized_frq'}  # Replace with the keys you want to remove

def remove_keys_from_dict(d, keys_to_remove):
    """Recursively remove specified keys from a nested dictionary."""
    if isinstance(d, dict):
        return {k: remove_keys_from_dict(v, keys_to_remove) for k, v in d.items() if k not in keys_to_remove}
    elif isinstance(d, list):
        return [remove_keys_from_dict(item, keys_to_remove) for item in d]
    else:
        return d

# Iterate over JSON files in the folder
for file_path in sorted(folder_path.glob("*.json")):
    if file_path.stat().st_size == 0:  # Skip empty files
        print(f"Skipping empty file: {file_path.name}")
        continue
    
    try:
        with open(file_path, "r") as f:
            data = json.load(f)  # Load JSON file

        # Remove specified keys
        cleaned_data = remove_keys_from_dict(data, keys_to_remove)

        # Save the cleaned JSON back to the file
        with open(file_path, "w") as f:
            json.dump(cleaned_data, f, indent=4)

        print(f"Processed: {file_path.name}")

    except json.JSONDecodeError:
        print(f"Skipping invalid JSON file: {file_path.name}")


Processed: case_1.json
Processed: case_10.json
Processed: case_100.json
Processed: case_101.json
Processed: case_102.json
Processed: case_103.json
Processed: case_104.json
Processed: case_106.json
Processed: case_107.json
Processed: case_109.json
Processed: case_11.json
Processed: case_112.json
Processed: case_113.json
Processed: case_115.json
Processed: case_117.json
Processed: case_118.json
Processed: case_119.json
Processed: case_12.json
Processed: case_121.json
Processed: case_123.json
Processed: case_124.json
Processed: case_126.json
Processed: case_127.json
Processed: case_128.json
Processed: case_13.json
Processed: case_131.json
Processed: case_132.json
Processed: case_134.json
Processed: case_135.json
Processed: case_137.json
Processed: case_139.json
Processed: case_14.json
Processed: case_140.json
Processed: case_141.json
Processed: case_143.json
Processed: case_144.json
Processed: case_145.json
Processed: case_146.json
Processed: case_147.json
Processed: case_148.json
Process